# Opening images in Keras
Todo:
 - Change from Pillow to rasterio to read image
 - 

Sources:
    - https://medium.com/@ensembledme/writing-custom-keras-generators-fe815d992c5a
    - https://stackoverflow.com/questions/46493419/use-a-generator-for-keras-model-fit-generator
    - https://stackoverflow.com/questions/50195967/how-can-i-create-a-4d-numpy-array-from-images
    - https://www.google.com/search?client=firefox-b-ab&q=keras+make+image+generator
 

In [2]:
from PIL import Image

import keras
from PIL import Image

import imageio
import random
import scipy
import numpy as np



In [2]:
import rasterio
import numpy as np

with rasterio.open('../Sorted_tiles/testimage.tif', 'r') as ds:
    arr = ds.read()  # read all raster values
    #arr = np.moveaxis(arr, 0, -1) # Moves the channel index to the back.

print(arr.shape) 

import imageio
im = imageio.imread('../Sorted_tiles/testimage.tif')
print(im.shape) 

RasterioIOError: ../Sorted_tiles/testimage.tif: No such file or directory

In [3]:
import glob
import os
list_images = glob.glob('../Sorted_tiles/*.tif')

batch_paths = np.random.choice(a = list_images, 
                                         size = 2)
print(batch_paths)
print(os.path.basename(batch_paths[0]))

['../Sorted_tiles/all_bands_merged_10_43.tif'
 '../Sorted_tiles/all_bands_merged_17_39.tif']
all_bands_merged_10_43.tif


In [4]:
k = np.random.randint(low=1, high=6,  size=30)
print(k)

[4 5 1 1 5 3 3 1 2 2 2 4 3 2 3 5 2 5 5 5 2 4 1 5 1 5 5 5 5 3]


## Read imagelabels

In [5]:
import pandas as pd
labelfile = pd.read_csv('../Qgis/labeled_tiles.csv')
labelfile.head()

,ID,Tile_name,LIVSM1,LIVSM2,mult_bin_labels,1,2,3,4,5,6,7,8,9,10,11,12,99
0,0,all_bands_merged_01_31.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,all_bands_merged_01_32.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,all_bands_merged_01_33.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,all_bands_merged_01_34.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,all_bands_merged_01_35.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1


In [13]:
labelfile.describe()

,ID,LIVSM1,LIVSM2,1,2,3,4,5,6,7,8,9,10,11,12,99
count,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.0,551.0,551.0,551.0,551.0,551.0,551.0,551.000000
mean,275.000000,92.520871,92.459165,0.025408,0.021779,0.016334,0.014519,0.019964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.932849
std,159.204271,24.174724,24.405321,0.157505,0.146092,0.126871,0.119726,0.140003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250510
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,137.500000,99.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
50%,275.000000,99.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
75%,412.500000,99.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
max,550.000000,99.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


## Homemade image data generator

In [8]:
def flip_vertical(image):
    return np.flipud(image)

def flip_horizontal(image):
    return np.fliplr(image)


def rotate_n_deg(image):
    # Rotates the image a random amount. Fills the space with a reflection of the image. Kva blir best her? Heller bruke rot90?
    rot = random.randrange(-180,180,30)
    rot_image = scipy.ndimage.interpolation.rotate(image, rot,axes=(1, 0), reshape=False, output=None, order=3, mode='reflect', cval=0.0, prefilter=True)

    return rot_image

#def rotate_180(image):
#    return np.rot90(image, k=2)

def random_image_flip_rotation(image):
    # This function rotates or flips the image randomly using one of five functions. 
    # This function can also do nothing to the image. 1/4 chanche.
    # Returns a image

    randint = np.random.randint(low=1, high=4)
    modified_image = 0
    
    if randint == 1:
        modified_image = flip_vertical(image)
    elif randint == 2:
        modified_image = flip_horizontal(image)

    elif randint == 3:
        modified_image = rotate_n_deg(image)

    else:
        modified_image = image
        
    return modified_image

A = np.array([[1, 2, 3], [4, 5, 6], [7,8,9]])
print(A)
print(flip_vertical(A))
print(flip_horizontal(A))
print(rotate_n_deg(A))
print(random_image_flip_rotation(A))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[7 8 9]
 [4 5 6]
 [1 2 3]]
[[3 2 1]
 [6 5 4]
 [9 8 7]]
[[7 6 3]
 [9 5 1]
 [7 4 4]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [9]:
list_images = glob.glob('../Sorted_tiles/*.tif')


def get_input(path):
    
    img = imageio.imread(path)
    
    return(img)


def image_generator(list_images, label_file, batch_size = 1):
    # list_images: list of where the images are placed.
    # label_file: csv file with labels and filenames.
    
    # Check that all images are the same size. Dimentions
    while True:
        batch_input = []
        batch_y = []
        batch_x = 0
        
        
        # Select files (paths/indices) for the batch
        batch_paths = np.random.choice(a = list_images, 
                                     size = batch_size)

        # Read in each input, perform preprocessing and get labels

        for input_path in batch_paths:

          input = get_input(input_path)
          batch_input += [ input ]

          # Set a label by using the tile name
          #label = labelfile[labelfile.Tile_name == os.path.basename(input_path)]['mult_bin_labels']
            
          # Set a label by using the tile name. This line first finds the rowdata for 
          # a spesific tile and gives it out as a dataframe. Then the .iloc function 
          # finds the values of the colums 1 up to 99 and give them out as a list(list in a list). 
          label = labelfile.loc[labelfile.Tile_name == os.path.basename(input_path), ['1','2','3','4','5','6','7','8','9','10','11','12','99']].values

        
        
          batch_y.append(label) # Needs to be int not string, is it working whitout?

        # Return a tuple of (input,output) to feed the network
        batch_x = np.stack(batch_input, axis=0)

        yield(batch_x, batch_y)

In [10]:
test_generated_images = image_generator(list_images, labelfile)
generert = next(test_generated_images)
print(generert)
#print(next(test_generated_images))
print(np.shape(generert[0]))
print(generert[1])

(array([[[[9.32935253e-03, 9.22781881e-03, 1.01920133e-02, ...,
          0.00000000e+00, 6.00743442e-05, 1.10718938e-04],
         [9.47490428e-03, 1.00847464e-02, 8.92224442e-03, ...,
          0.00000000e+00, 6.00743442e-05, 1.10718938e-04],
         [7.87383970e-03, 8.15666001e-03, 9.07615572e-03, ...,
          0.00000000e+00, 6.00743442e-05, 1.10718938e-04],
         ...,
         [9.16397106e-03, 9.08463262e-03, 1.02219190e-02, ...,
          1.45523925e-04, 1.01651221e-06, 8.37169719e-05],
         [9.31070466e-03, 8.91138613e-03, 1.10745849e-02, ...,
          1.45523925e-04, 1.01651221e-06, 8.37169719e-05],
         [8.13683961e-03, 9.21456795e-03, 8.12901091e-03, ...,
          3.05185204e-05, 8.58881904e-05, 2.26979901e-04]],

        [[1.02689974e-02, 8.16250499e-03, 9.76702478e-03, ...,
          0.00000000e+00, 6.00743442e-05, 1.10718938e-04],
         [9.83680040e-03, 9.91861895e-03, 9.72889643e-03, ...,
          0.00000000e+00, 6.00743442e-05, 1.10718938e-04],
       

/home/landsat7/anaconda3/envs/master2018/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_packbits'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/landsat7/anaconda3/envs/master2018/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_lzw'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/landsat7/anaconda3/envs/master2018/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'unpack_ints'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/landsat7/anaconda3/envs/master2018/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'reverse_bitorder'
  Functiona

In [11]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 50, 474)))
model.add(Activation('sigmoid'))

model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(13, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        136544    
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8448      
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)                3341      
Total para

In [24]:
model.fit_generator(
        image_generator(list_images, labelfile, batch_size = 1),
        steps_per_epoch=50,
        epochs=10)

Epoch 1/10
50/50 [==============================] - 7s 131ms/step - loss: 0.0425 - acc: 0.9954
Epoch 2/10
50/50 [==============================] - 6s 117ms/step - loss: 0.1448 - acc: 0.9831
Epoch 3/10
50/50 [==============================] - 6s 117ms/step - loss: 0.1159 - acc: 0.9877
Epoch 4/10
50/50 [==============================] - 6s 111ms/step - loss: 0.0489 - acc: 0.9938
Epoch 5/10
 3/50 [>.............................] - ETA: 5s - loss: 0.7453 - acc: 0.9231   

KeyboardInterrupt: 

In [20]:
nn = model.predict_generator(
        image_generator(list_images, labelfile, batch_size = 1),
        steps=5)
print(nn)

TypeError: predict_generator() got an unexpected keyword argument 'validation'

In [19]:
#img = Image.open('../Sorted_tiles/kkkk.tiff')

from osgeo import gdal
import numpy as np
ds = gdal.Open('../Sorted_tiles/testimage.tif')
myarray = np.array(ds.GetRasterBand(1).ReadAsArray())

()


In [ ]:
# Finne ut korleis eg får lest det slik at matrisa blir 4 dimensjonal.
# Check that all images are the same size. Dimentions